# Linear regression Medical Insurance

### Import Libraries

In [104]:
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Feature Selection
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Model Building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Evaluation Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# To avoid warning
import warnings
warnings.filterwarnings("ignore")

import pickle
import json

# Problem Statement

In [105]:
#To predict charges for medical insurance.

# Data Gathering

In [106]:
df = pd.read_csv("medical_insurance.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [108]:
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [109]:
df['sex'].value_counts()

sex
male      676
female    662
Name: count, dtype: int64

In [110]:
df['sex'].replace({"female":0, "male":1}, inplace= True)

In [111]:
sex_value = {"female":0, "male":1}
sex_value

{'female': 0, 'male': 1}

In [112]:
df['sex'].value_counts()

sex
1    676
0    662
Name: count, dtype: int64

In [113]:
df['smoker'].value_counts()

smoker
no     1064
yes     274
Name: count, dtype: int64

In [114]:
df['smoker'].replace({"no":0, "yes":1}, inplace= True)

In [115]:
smoker_value = {"no":0, "yes":1}
smoker_value

{'no': 0, 'yes': 1}

In [116]:
df['smoker'].value_counts()

smoker
0    1064
1     274
Name: count, dtype: int64

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   int64  
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   int64  
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 73.3+ KB


In [118]:
df['region'].value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

In [119]:
df = pd.get_dummies(df, columns=['region'])
df

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,False,False,False,True
1,18,1,33.770,1,0,1725.55230,False,False,True,False
2,28,1,33.000,3,0,4449.46200,False,False,True,False
3,33,1,22.705,0,0,21984.47061,False,True,False,False
4,32,1,28.880,0,0,3866.85520,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830,False,True,False,False
1334,18,0,31.920,0,0,2205.98080,True,False,False,False
1335,18,0,36.850,0,0,1629.83350,False,False,True,False
1336,21,0,25.800,0,0,2007.94500,False,False,False,True


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   sex               1338 non-null   int64  
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   int64  
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   bool   
 7   region_northwest  1338 non-null   bool   
 8   region_southeast  1338 non-null   bool   
 9   region_southwest  1338 non-null   bool   
dtypes: bool(4), float64(2), int64(4)
memory usage: 68.1 KB


# Model Building

In [121]:
x = df.drop('charges', axis= 1)
y = df['charges']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=9)
x_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
333,56,0,28.785,0,0,True,False,False,False
644,43,1,35.310,2,0,False,False,True,False
1134,28,1,33.820,0,0,False,True,False,False
852,46,0,35.530,0,1,True,False,False,False
514,39,1,28.300,1,1,False,False,False,True
...,...,...,...,...,...,...,...,...,...
56,58,0,31.825,2,0,True,False,False,False
501,43,1,26.030,0,0,True,False,False,False
638,39,1,26.410,0,1,True,False,False,False
348,36,0,29.920,1,0,False,False,True,False


In [122]:
# create an instance of model

model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [123]:
# Model Evaluation for Testing Data
y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Sqaured Error :",mse)

rmse = np.sqrt(mse)
print("Root Mean Sqaured Error :",rmse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error :",mae)

r_squared_value = r2_score(y_test, y_pred)
print("R Squared Value :",r_squared_value)

adj_r2  = 1 - (((1 - r_squared_value) * (x_test.shape[0] - 1)) / (x_test.shape[0] - x_test.shape[1] - 1))
print("Adjusted R Squared Value :",adj_r2)

Mean Sqaured Error : 31450998.635947134
Root Mean Sqaured Error : 5608.118992670104
Mean Absolute Error : 3820.1665530724285
R Squared Value : 0.7775428684674436
Adjusted R Squared Value : 0.7697827359721219


In [124]:
# Model Evaluation for Training Data

y_pred_train = model.predict(x_train)

mse = mean_squared_error(y_train, y_pred_train)
print("Mean Sqaured Error :",mse)

rmse = np.sqrt(mse)
print("Root Mean Sqaured Error :",rmse)

mae = mean_absolute_error(y_train, y_pred_train)
print("Mean Absolute Error :",mae)

r_squared_value = r2_score(y_train, y_pred_train)
print("R Squared Value :",r_squared_value)

adj_r2  = 1 - (((1 - r_squared_value) * (x_train.shape[0] - 1)) / (x_train.shape[0] - x_train.shape[1] - 1))
print("Adjusted R Squared Value :",adj_r2)

Mean Sqaured Error : 37822202.96540095
Root Mean Sqaured Error : 6149.975850798192
Mean Absolute Error : 4245.9796818395
R Squared Value : 0.7441475882031922
Adjusted R Squared Value : 0.7419752564049173


In [125]:
x.head(1).T

,0
age,19
sex,0
bmi,27.9
children,0
smoker,1
region_northeast,False
region_northwest,False
region_southeast,False
region_southwest,True


In [126]:
columns_names=list(x.columns)
columns_names

['age',
 'sex',
 'bmi',
 'children',
 'smoker',
 'region_northeast',
 'region_northwest',
 'region_southeast',
 'region_southwest']

In [127]:
with open ("Linear_model.pkl","wb") as f:
    pickle.dump(model,f)

In [128]:
json_data={"sex":sex_value,"smoker":smoker_value,"columns":columns_names}
json_data

{'sex': {'female': 0, 'male': 1},
 'smoker': {'no': 0, 'yes': 1},
 'columns': ['age',
  'sex',
  'bmi',
  'children',
  'smoker',
  'region_northeast',
  'region_northwest',
  'region_southeast',
  'region_southwest']}

In [129]:
with open ("json_data.json","w") as f:
    json.dump(json_data,f)

In [130]:
#Testing on Single Use Input

In [131]:
age= 35
sex= 'male'
bmi= 24.5
children= 2
smoker= 'yes'
region= 'northeast'
region= 'region_' + region

In [132]:
region

'region_northeast'

In [133]:
len(columns_names)

9

In [134]:
# region_index=np.where(json_data["columns"] == region)
# region_index

In [135]:
# region_index= np.where(columns_names==region)[0]  #Will not work since coumns_names is in list and not array
# region_index

In [136]:
region_index=columns_names.index(region)
region_index

5

In [137]:
test_array=np.zeros(len(columns_names))
test_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [138]:
test_array[0]=age
test_array[1]=json_data["sex"][sex]
test_array[2]=bmi
test_array[3]=children
test_array[4]=json_data["smoker"][smoker]
test_array[region_index]=1

In [139]:
test_array

array([35. ,  1. , 24.5,  2. ,  1. ,  1. ,  0. ,  0. ,  0. ])

In [140]:
charges=model.predict([test_array])
charges[0]

29789.640331904397

In [141]:
print(f"Insurance charges are Rs {charges[0]}/-")

Insurance charges are Rs 29789.640331904397/-


In [142]:
#  def addition(a,b):
#      a=5
#      b=7
#      sum=a+b
#      print(f"addition of {a} and {b} is {sum}")
#      return sum

In [143]:
# addition(6,7)

In [144]:
# b=10
# a=addition(2,4)
# a
# print(b)

In [145]:
# a=45
# print(a)
# print(b)